In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Header tags from wikipedia main page

In [2]:
page=requests.get('https://en.wikipedia.org/wiki/Main_Page')
soup=BeautifulSoup(page.content,'html.parser')
tags=soup.find_all(['h1','h2','h3','h4','h5','h6'])
tag=[]
for i in tags:
    tag.append(i)
tags=pd.DataFrame({})
tags['Tag']=tag
tags.to_csv('wiki_tags')

# IMDB top moves page 1 and page 2

In [3]:
def scrape(url):
    movie_name=[]
    year=[]
    rating=[]
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    titles=soup.find_all('h3',class_='lister-item-header')
    for i in titles:
        for j in i.find_all('a'):
            movie_name.append(j.text)
    
    years=soup.find_all('span',class_='lister-item-year text-muted unbold')   
    for i in years:
        year.append(i.text)
    
    ratings=soup.find_all('div',class_='inline-block ratings-imdb-rating')
    for i in ratings:
        for j in i.find_all('strong'):
            rating.append(j.text)
    
    movies=pd.DataFrame({})
    movies['Name']=movie_name
    movies['Year']=year
    movies['rating']=rating
    movies.head()
    return movies

In [4]:
scrape_pg1=scrape('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc')
scrape_pg2=scrape('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc&start=51&ref_=adv_nxt')
(scrape_pg1.append(scrape_pg2,ignore_index=True )).to_csv('movie_list')

# Indian movies

In [5]:
def scrape_indian(url):
    movie_name=[]
    year=[]
    rating=[]
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    titles=soup.find_all('h3',class_='lister-item-header')
    for i in titles:
        for j in i.find_all('a'):
            movie_name.append(j.text)
    
    years=soup.find_all('span',class_='lister-item-year text-muted unbold')   
    for i in years:
        year.append(i.text)
    
    ratings=soup.find_all('div',class_='ipl-rating-star small')
    for i in ratings:
        for j in i.find_all('span',class_='ipl-rating-star__rating'):
            rating.append(j.text)
    
    movies_ind=pd.DataFrame({})
    movies_ind['Name']=movie_name
    movies_ind['Year']=year
    movies_ind['rating']=rating
    return movies_ind
res=scrape_indian('https://www.imdb.com/list/ls056092300/')
res.to_csv('top100indian')

# book reviews 

In [6]:
def book_info(url):
    book_name=[]
    author=[]
    genre=[]
    review=[]
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
    names=soup.find_all('h4',class_='italic')
    for i in names:
        book_name.append(i.text.replace('\n',''))
    authors=soup.find_all('p',class_='sans bold')
    for i in authors:
        author.append(i.text.replace('\n',''))
    genres=soup.find_all('p',class_='genre-links hidden-phone')
    for i in genres:
        genre.append(i.text.replace('\n',''))
    reviews=soup.find_all('p',class_='excerpt')
    for i in reviews:
        review.append(i.text.replace('\n',''))
    books=pd.DataFrame({})
    books['Name']=book_name
    books['Author']=author
    books['Genre']=genre
    books['Review']=review
    return books
(book_info('https://bookpage.com/reviews')).to_csv('Book Reviews')

# san francisco weather 

In [7]:
time=[]
forecast=[]
page=requests.get('https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YAAeuugzaUk')
soup=BeautifulSoup(page.content,'html.parser')
titles=soup.find_all('div',class_='col-sm-2 forecast-label')
for i in titles:
    time.append(i.get_text())
forecasts=soup.find_all('div',class_='col-sm-10 forecast-text')
for i in forecasts:
    forecast.append(i.get_text())
weather=pd.DataFrame({})
weather['Time']=time
weather['Forecast']=forecast
weather.to_csv('Weather')

# phones under 20k

In [8]:
import selenium
from selenium import webdriver

In [9]:
driver = webdriver.Chrome(r'chromedriver.exe')
url='https://www.amazon.in/s?k=mobile+phones+under+20000&rh=p_36%3A-2000000&page=1&crid=K6WMQVL9S76S&qid=1610219994&rnid=1318502031&sprefix=mobile+phones+%2Caps%2C288'
driver.get(url)
import time
from selenium.common.exceptions import NoSuchElementException
j=0
urls=[]
#scraping only 5 pages as there are almost 315 results and scraping one by one will take a lot of time
while j<5:
    time.sleep(5)
    url_tags=driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')
    for url in url_tags:
        i=url.get_attribute('href')
        urls.append(i)
    
    driver.find_element_by_xpath('//li[@class="a-last"]').click()
    j+=1
    print(j)
name=[]
price=[]
img_link=[]
for url in urls:
    driver.get(url)
    try:
        title=driver.find_element_by_xpath('//span[@class="a-size-large product-title-word-break"]')
        name.append(title.text)
    except NoSuchElementException:
        name.append('--')
    try:
        cost=driver.find_element_by_xpath('//span[@class="a-size-medium a-color-price priceBlockBuyingPriceString"]')
        price.append(cost.text)
    except NoSuchElementException:
        price.append('--')
    try:
        img=driver.find_element_by_xpath('//img[@class="imgTagWrapper"]')
        img_link.append(img.get_attribute('src'))
    except:
        img_link.append('NA')
phones=pd.DataFrame({})
phones['Name']=name
phones['Price']=price
phones['Image_Link']=img_link
phones.to_csv('Phones_under_20k')

1
2
3
4
5


# monster software developer jobs

In [10]:
driver = webdriver.Chrome(r'chromedriver.exe')
url='https://www.monsterindia.com/srp/results?query=software%20developers&searchId=dd5f0cbd-b182-439c-ac5b-dece7607a9bd'
driver.get(url)
job_title=[]
company=[]
location=[]
j=0
while j<5:
    time.sleep(5)
    titles=driver.find_elements_by_xpath('//h3[@class="medium"]')
    for i in titles:
        job_title.append(i.text)
    
    companies=driver.find_elements_by_xpath('//span[@class="company-name"]')
    for i in titles:
        company.append(i.text)
    
    locations=driver.find_elements_by_xpath('//span[@class="loc"]')
    for i in titles:
        location.append(i.text)
    driver.find_element_by_xpath('//button[@class="btn-next-prev btn-next"]').click()
    j+=1
jobs_monster=pd.DataFrame({})
jobs_monster['JobTitle']=job_title
jobs_monster['Company']=company

jobs_monster.to_csv('monster_soft_developer')

# monster delhi jobs

In [11]:
driver = webdriver.Chrome(r'chromedriver.exe')
url='https://www.monsterindia.com/srp/results?query=Data%20Scientist&locations=Delhi&searchId=4b2da987-8b2b-480f-bbe5-cbd662f00c04'
driver.get(url)
job_title=[]
company=[]
location=[]
j=0
while j<5:
    time.sleep(5)
    titles=driver.find_elements_by_xpath('//h3[@class="medium"]')
    for i in titles:
        job_title.append(i.text)
    
    companies=driver.find_elements_by_xpath('//span[@class="company-name"]')
    for i in companies:
        company.append(i.text)
    
    locations=driver.find_elements_by_xpath('//span[@class="loc"]')
    for i in locations:
        location.append(i.text)
    driver.find_element_by_xpath('//button[@class="btn-next-prev btn-next"]').click()
    j+=1
jobs_monster=pd.DataFrame({})
jobs_monster['JobTitle']=job_title
jobs_monster['Company']=company[0:125]

jobs_monster.to_csv('monster_DS_delhi')